In [ ]:
# from google.colab import auth
# from googleapiclient.discovery import build

# # Authenticate as yourself
# auth.authenticate_user()

# # Build Sheets API client
# service = build("sheets", "v4")

# SPREADSHEET_ID = "12UHiKYqrNKc-Ppo296pBCS6uTCmT2Vsk6Q1eIPT1--M"

# # Fetch calendar grid with formatting
# result = service.spreadsheets().get(
#     spreadsheetId=SPREADSHEET_ID,
#     ranges=["2025!B1:M31"],
#     includeGridData=True
# ).execute()

# cells = result["sheets"][0]["data"][0]["rowData"]


In [ ]:
# ============================================================
# Mood Dashboard Extractor (Google Sheets API)
# - Reads cell background colors from your mood calendars
# - Maps colors -> (emotion, score)
# - Builds a tidy dataset across:
#     * 2023, 2024, 2025 (single-year tabs, range B2:M31)
#     * 2020–2022 composed from the bridge tabs:
#         - "2020-2021" and "2021-2022"
# - Outputs:
#     * df_all (all extracted days)
#     * monthly_hi
#     * year_emotion breakdown
#     * Total Emotional Index (TEI)
# ============================================================

# Import Libraries and Auth

In [2]:
!pip -q install --upgrade google-api-python-client google-auth

In [3]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
import pandas as pd
from datetime import date
import google.auth

# Read-only is enough (safer)
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

creds, _ = google.auth.default(scopes=SCOPES)
service = build("sheets", "v4", credentials=creds)


# Configs

In [4]:
SPREADSHEET_ID = "12UHiKYqrNKc-Ppo296pBCS6uTCmT2Vsk6Q1eIPT1--M"
  # <-- REQUIRED

# Your year tabs (single-year)
SINGLE_YEAR_TABS = {
    "2025": {"range": "B2:M32", "year": 2025},
    "2024": {"range": "B2:M32", "year": 2024},
    "2023": {"range": "B2:M32", "year": 2023},
}

# Your bridge tabs (multi-year)
BRIDGE_TABS = {
    "2020-2021": {"range": "B2:N32"},  # columns correspond to: Oct 2020 ... Oct 2021
    "2021-2022": {"range": "B2:O32"},  # columns correspond to: Nov 2021 ... Dec 2022
}

RANGE_OVERRIDES = {
    # "2020-2021": "B3:N32",
    # "2021-2022": "B3:O32",
    # "2023": "B3:M32",
    # "2024": "B3:M32",
    # "2025": "B3:M32",
}

# Emotion legend (label - score - "palette name")
EMOTIONS = [
    ("Happy", 11, "yellow"),
    ("Productive", 10, "dark_green"),
    ("Good", 9, "light_green"),
    ("Tired", 8, "light_blue"),
    ("Lazy", 7, "pink"),
    ("SAD", 6, "grey"),
    ("Stress/Anxiety", 5, "orange"),
    ("Angry", 4, "red"),
    ("Depressed", 3, "dark_blue"),
    ("Hopeless", 2, "purple"),
    ("Suicidal", 1, "black"),
]

# Approx RGB palette (0..1 floats). We "snap" actual sheet colors to these.
# If you see unmapped colors, we'll print them so you can refine this palette.
PALETTE_HEX = {
    "yellow":      "#FFFF00",
    "dark_green":  "#008000",
    "light_green": "#90EE90",
    "light_blue":  "#ADD8E6",
    "pink":        "#FFC0CB",
    "grey":        "#808080",
    "orange":      "#FFA500",
    "red":         "#FF0000",
    "dark_blue":   "#00008B",
    "purple":      "#800080",
    "black":       "#000000",
}

# Tolerance for "nearest match" in RGB-int space (0..255)
HEX_SNAP_THRESHOLD = 40  # tune if needed (e.g., 25 stricter, 60 more forgiving)



# Helpers

In [17]:
# ============================================================
# 2) Low-level helpers (fetch grid, read colors as HEX, map via LEGEND)
# ============================================================

def get_effective_range(sheet_name: str, default_range: str):
    """
    Return the range to use for a sheet.
    Allows per-sheet overrides via RANGE_OVERRIDES.
    """
    return RANGE_OVERRIDES.get(sheet_name, default_range)

def quote_sheet_name(name: str) -> str:
    safe = name.replace("'", "''")
    return f"'{safe}'"

def fetch_grid_rowdata(sheet_name: str, a1_range: str):
    full_range = f"{quote_sheet_name(sheet_name)}!{a1_range}"
    resp = service.spreadsheets().get(
        spreadsheetId=SPREADSHEET_ID,
        ranges=[full_range],
        includeGridData=True
    ).execute()
    return resp["sheets"][0]["data"][0].get("rowData", [])

def rgb_float_to_hex(r: float, g: float, b: float) -> str:
    return "#{:02X}{:02X}{:02X}".format(
        int(round(r * 255)),
        int(round(g * 255)),
        int(round(b * 255)),
    )

def hex_to_rgb_int(hex_color: str):
    """'#RRGGBB' -> (R,G,B) ints in [0,255]."""
    h = hex_color.lstrip("#")
    return (int(h[0:2], 16), int(h[2:4], 16), int(h[4:6], 16))

def color_distance_rgb_int(c1, c2) -> float:
    """Euclidean distance in RGB-int space."""
    return ((c1[0]-c2[0])**2 + (c1[1]-c2[1])**2 + (c1[2]-c2[2])**2) ** 0.5


def hex_from_cell(cell: dict):
    """
    Extract background fill as HEX '#RRGGBB' or None.

    Key fix:
      Black (#000000) is often returned by the API as an empty rgb dict {} or
      as a backgroundColor/backgroundColorStyle present but with missing channels.
      We treat "present but empty" as black, not None.

    We also check userEnteredFormat first since you manually fill cells.
    """

    def rgb_dict_to_hex(rgb: dict):
        # If the dict is present but empty, interpret as BLACK
        # (Sheets often omits channels for black)
        if rgb == {}:
            return "#000000"

        # Missing channels default to 0.0 (not 1.0)
        r = float(rgb.get("red", 0.0))
        g = float(rgb.get("green", 0.0))
        b = float(rgb.get("blue", 0.0))

        # Pure white means empty
        if r == 1.0 and g == 1.0 and b == 1.0:
            return None

        return "#{:02X}{:02X}{:02X}".format(
            int(round(r * 255)),
            int(round(g * 255)),
            int(round(b * 255)),
        )

    # 1) Prefer userEnteredFormat (manual fills)
    uef = cell.get("userEnteredFormat") or {}

    u_style = uef.get("backgroundColorStyle") or {}
    u_rgb = u_style.get("rgbColor")
    if isinstance(u_rgb, dict):
        hx = rgb_dict_to_hex(u_rgb)
        if hx is not None:
            return hx

    u_bg = uef.get("backgroundColor")
    if isinstance(u_bg, dict):
        hx = rgb_dict_to_hex(u_bg)
        if hx is not None:
            return hx

    # 2) Fallback to effectiveFormat
    fmt = cell.get("effectiveFormat") or {}

    e_style = fmt.get("backgroundColorStyle") or {}
    e_rgb = e_style.get("rgbColor")
    if isinstance(e_rgb, dict):
        hx = rgb_dict_to_hex(e_rgb)
        if hx is not None:
            return hx

    e_bg = fmt.get("backgroundColor")
    if isinstance(e_bg, dict):
        hx = rgb_dict_to_hex(e_bg)
        if hx is not None:
            return hx

    return None


In [18]:
# -----------------------------
# LEGEND RANGES (EDIT THESE)
# -----------------------------
LEGEND_SHEET = "2023"
LEGEND_COLOR_RANGE = "N2:N12"  # colored cells
LEGEND_LABEL_RANGE = "O2:O12"  # labels
LEGEND_SCORE_RANGE = "P2:P12"  # scores

def fetch_values(sheet_name: str, a1_range: str):
    full_range = f"{quote_sheet_name(sheet_name)}!{a1_range}"
    resp = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=full_range
    ).execute()
    return resp.get("values", [])

def build_legend_map():
    """
    Returns:
      LEGEND_MAP: hex -> {"emotion": str, "score": int}
      LEGEND_RGB_INT: hex -> (R,G,B)
    """
    color_rows = fetch_grid_rowdata(LEGEND_SHEET, LEGEND_COLOR_RANGE)
    labels = fetch_values(LEGEND_SHEET, LEGEND_LABEL_RANGE)
    scores = fetch_values(LEGEND_SHEET, LEGEND_SCORE_RANGE)

    legend = {}
    for i, r in enumerate(color_rows):
        vals = r.get("values", [])
        if not vals:
            continue

        hx = hex_from_cell(vals[0])
        if not hx:
            continue

        label = labels[i][0].strip() if i < len(labels) and labels[i] else ""
        score = int(scores[i][0]) if i < len(scores) and scores[i] else None

        if label and score is not None:
            legend[hx] = {"emotion": label, "score": score}

    legend_rgb = {hx: hex_to_rgb_int(hx) for hx in legend.keys()}
    return legend, legend_rgb

LEGEND_MAP, LEGEND_RGB_INT = build_legend_map()
print("Legend entries:", len(LEGEND_MAP))
print(LEGEND_MAP)

# Distance threshold for "nearest legend color" fallback
# (Increase slightly if you still see UNMAPPED days)
LEGEND_SNAP_THRESHOLD = 80

def map_hex_via_legend(hex_color: str):
    """
    Map a day cell hex to emotion/score using:
      1) exact match
      2) nearest legend color (fallback)
    Returns: (emotion, score, palette_match, match_dist)
    """
    if hex_color in LEGEND_MAP:
        item = LEGEND_MAP[hex_color]
        return item["emotion"], int(item["score"]), "LEGEND_EXACT", 0.0

    # Nearest neighbor to legend colors
    obs = hex_to_rgb_int(hex_color)
    best = None  # (hx, dist)

    for hx, target in LEGEND_RGB_INT.items():
        d = color_distance_rgb_int(obs, target)
        if best is None or d < best[1]:
            best = (hx, d)

    if best and best[1] <= LEGEND_SNAP_THRESHOLD:
        hx_best, d = best
        item = LEGEND_MAP[hx_best]
        return item["emotion"], int(item["score"]), "LEGEND_NEAREST", float(d)

    return "UNMAPPED", None, None, None

Legend entries: 11
{'#FFD966': {'emotion': 'Happy', 'score': 11}, '#38761D': {'emotion': 'Productive', 'score': 10}, '#93C47D': {'emotion': 'Good', 'score': 9}, '#9FC5E8': {'emotion': 'Tired', 'score': 8}, '#EAD1DC': {'emotion': 'Lazy', 'score': 7}, '#B7B7B7': {'emotion': 'SAD', 'score': 6}, '#D1802C': {'emotion': 'Stress/Anxiety', 'score': 5}, '#CC0000': {'emotion': 'Angry/Annoyed', 'score': 4}, '#1155CC': {'emotion': 'Depressed', 'score': 3}, '#674EA7': {'emotion': 'Hopeless', 'score': 2}, '#000000': {'emotion': 'Suicidal', 'score': 1}}


# Extractor 1: for single-year tabs

In [19]:
# ============================================================
# 3) Extractor for single-year tabs (2023–2025)
# ============================================================

def extract_single_year_fixed(sheet_name: str, grid_range: str, year: int):
    rowData = fetch_grid_rowdata(sheet_name, grid_range)

    rows = []
    unmapped = set()

    for r_idx, row in enumerate(rowData):
        day = r_idx + 1
        vals = row.get("values", [])

        for c_idx, cell in enumerate(vals):
            month = c_idx + 1  # assumes 12 columns Jan..Dec

            hex_color = hex_from_cell(cell)
            if hex_color is None:
                # If you truly fill every cell, this should only happen on actual whites
                continue

            emotion, score, palette_name, d = map_hex_via_legend(hex_color)

            if emotion == "UNMAPPED":
                unmapped.add(hex_color)

            # skip invalid dates (Feb 30 etc.)
            try:
                dt = date(year, month, day)
            except ValueError:
                continue

            rows.append({
                "date": pd.to_datetime(dt),
                "year": year,
                "month": month,
                "day": day,
                "sheet": sheet_name,
                "emotion": emotion,
                "score": int(score) if score is not None else None,
                "color_hex": hex_color,
                "palette_match": palette_name,
                "match_dist": float(d) if d is not None else None,
            })

    df = pd.DataFrame(rows).sort_values("date").reset_index(drop=True)
    return df, unmapped


# Extractor 2: for bridged tabs

In [20]:
# ============================================================
# 4) Extractors for bridge tabs (2020–2022 composition)
# ============================================================

def extract_month_from_cached_grid(sheet_name: str, cached_rowData, year: int, month: int, grid_col_index_1based: int):
    rows = []
    unmapped = set()

    col_i = grid_col_index_1based - 1

    for r_idx, row in enumerate(cached_rowData):
        day = r_idx + 1
        vals = row.get("values", [])
        if col_i >= len(vals):
            continue

        cell = vals[col_i]

        hex_color = hex_from_cell(cell)
        if hex_color is None:
            continue

        emotion, score, palette_name, d = map_hex_via_legend(hex_color)

        if emotion == "UNMAPPED":
            unmapped.add(hex_color)

        try:
            dt = date(year, month, day)
        except ValueError:
            continue

        rows.append({
            "date": pd.to_datetime(dt),
            "year": year,
            "month": month,
            "day": day,
            "sheet": sheet_name,
            "emotion": emotion,
            "score": int(score) if score is not None else None,
            "color_hex": hex_color,
            "palette_match": palette_name,
            "match_dist": float(d) if d is not None else None,
        })

    return pd.DataFrame(rows), unmapped


def extract_composed_years_2020_2022():
    # IMPORTANT: use the exact tab names you actually have (with quotes handled by quote_sheet_name)
    range_2020_2021 = get_effective_range("2020-2021", BRIDGE_TABS["2020-2021"]["range"])
    range_2021_2022 = get_effective_range("2021-2022", BRIDGE_TABS["2021-2022"]["range"])

    grid_2020_2021 = fetch_grid_rowdata("2020-2021", range_2020_2021)
    grid_2021_2022 = fetch_grid_rowdata("2021-2022", range_2021_2022)

    unmapped_all = {"2020-2021": set(), "2021-2022": set()}

    # ---- 2020: Oct, Nov, Dec from 2020-2021 (cols 1..3)
    parts_2020 = []
    for month, col in [(10, 1), (11, 2), (12, 3)]:
        df_m, um = extract_month_from_cached_grid("2020-2021", grid_2020_2021, 2020, month, col)
        parts_2020.append(df_m)
        unmapped_all["2020-2021"] |= um
    df_2020 = pd.concat(parts_2020, ignore_index=True).sort_values("date").reset_index(drop=True)

    # ---- 2021:
    parts_2021 = []
    for month, col in [(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(7,10),(8,11),(9,12),(10,13)]:
        df_m, um = extract_month_from_cached_grid("2020-2021", grid_2020_2021, 2021, month, col)
        parts_2021.append(df_m)
        unmapped_all["2020-2021"] |= um

    for month, col in [(11,1),(12,2)]:
        df_m, um = extract_month_from_cached_grid("2021-2022", grid_2021_2022, 2021, month, col)
        parts_2021.append(df_m)
        unmapped_all["2021-2022"] |= um

    df_2021 = pd.concat(parts_2021, ignore_index=True).sort_values("date").reset_index(drop=True)

    # ---- 2022:
    parts_2022 = []
    mapping_2022 = [(1,3),(2,4),(3,5),(4,6),(5,7),(6,8),(7,9),(8,10),(9,11),(10,12),(11,13),(12,14)]
    for month, col in mapping_2022:
        df_m, um = extract_month_from_cached_grid("2021-2022", grid_2021_2022, 2022, month, col)
        parts_2022.append(df_m)
        unmapped_all["2021-2022"] |= um

    df_2022 = pd.concat(parts_2022, ignore_index=True).sort_values("date").reset_index(drop=True)

    return df_2020, df_2021, df_2022, unmapped_all


## Test if the system is working

In [ ]:
test = service.spreadsheets().get(
    spreadsheetId=SPREADSHEET_ID,
    ranges=["'2020-2021'!B2:N32"],
    includeGridData=True
).execute()

print("OK, got sheets:", len(test["sheets"]))


OK, got sheets: 1


In [ ]:
meta = service.spreadsheets().get(
    spreadsheetId=SPREADSHEET_ID,
    fields="sheets(properties(title))"
).execute()

titles = [s["properties"]["title"] for s in meta["sheets"]]
for t in titles:
    print(repr(t))


'2025'
'Daily Apps'
'2024'
'$$$'
'Sheet16'
'Productivity Report'
'2023'
'2021-2022'
'MoodDev-dataset'
'MoodDev-dataset(sample)'
'2020-2021'
'Months Ranked'
'David Burns Scores'
'Sheet15'


In [ ]:
test = service.spreadsheets().get(
    spreadsheetId=SPREADSHEET_ID,
    ranges=["'2023'!B14:B14"],   # change to a cell you KNOW is colored
    includeGridData=True
).execute()

cell = test["sheets"][0]["data"][0]["rowData"][0]["values"][0]
fmt = cell.get("effectiveFormat", {})

print("backgroundColor:", fmt.get("backgroundColor"))
print("backgroundColorStyle:", fmt.get("backgroundColorStyle"))
print("hex_from_cell:", hex_from_cell(cell))


backgroundColor: {'red': 0.8}
backgroundColorStyle: {'rgbColor': {'red': 0.8}}
hex_from_cell: #CC0000


In [ ]:
test = service.spreadsheets().get(
    spreadsheetId=SPREADSHEET_ID,
    ranges=["'2023'!B14:B14"],
    includeGridData=True
).execute()

cell = test["sheets"][0]["data"][0]["rowData"][0]["values"][0]
print("hex_from_cell:", hex_from_cell(cell))


hex_from_cell: #CC0000


In [ ]:
test = service.spreadsheets().get(
    spreadsheetId=SPREADSHEET_ID,
    ranges=["'2023'!B14:B14"],  # change sheet name if needed
    includeGridData=True
).execute()

cell = test["sheets"][0]["data"][0]["rowData"][0]["values"][0]
fmt = cell.get("effectiveFormat", {})

import json
print(json.dumps(fmt, indent=2))


{
  "backgroundColor": {
    "red": 0.8
  },
  "borders": {
    "top": {
      "style": "SOLID",
      "width": 1,
      "color": {},
      "colorStyle": {
        "rgbColor": {}
      }
    },
    "left": {
      "style": "SOLID",
      "width": 1,
      "color": {},
      "colorStyle": {
        "rgbColor": {}
      }
    },
    "right": {
      "style": "SOLID",
      "width": 1,
      "color": {},
      "colorStyle": {
        "rgbColor": {}
      }
    }
  },
  "padding": {
    "top": 2,
    "right": 3,
    "bottom": 2,
    "left": 3
  },
  "horizontalAlignment": "RIGHT",
  "verticalAlignment": "BOTTOM",
  "wrapStrategy": "OVERFLOW_CELL",
  "textFormat": {
    "foregroundColor": {},
    "fontFamily": "Arial",
    "fontSize": 10,
    "bold": false,
    "italic": false,
    "strikethrough": false,
    "underline": false,
    "foregroundColorStyle": {
      "rgbColor": {}
    }
  },
  "backgroundColorStyle": {
    "rgbColor": {
      "red": 0.8
    }
  }
}


# Run Extraction

In [21]:
# ============================================================
# Run Extraction (REBUILD df_all)
# ============================================================

df_2020, df_2021, df_2022, unmapped_bridge = extract_composed_years_2020_2022()

dfs_single = []
unmapped_single = {}

for tab, cfg in SINGLE_YEAR_TABS.items():
    grid_range = get_effective_range(tab, cfg["range"])
    df_y, um = extract_single_year_fixed(tab, grid_range, cfg["year"])
    dfs_single.append(df_y)
    if um:
        unmapped_single[tab] = um

df_2023_2025 = pd.concat(dfs_single, ignore_index=True)

df_all = (
    pd.concat([df_2020, df_2021, df_2022, df_2023_2025], ignore_index=True)
    .sort_values("date")
    .reset_index(drop=True)
)

print("Extracted total rows:", len(df_all))
print("Unique extracted days:", df_all["date"].nunique())
print(df_all.groupby("year")["date"].nunique())

# Optional: inspect how many UNMAPPED remain
print("UNMAPPED rows:", (df_all["emotion"] == "UNMAPPED").sum())

display(df_all.head(15))


Extracted total rows: 1899
Unique extracted days: 1899
year
2020     92
2021    365
2022    365
2023    365
2024    366
2025    346
Name: date, dtype: int64
UNMAPPED rows: 0


,date,year,month,day,sheet,emotion,score,color_hex,palette_match,match_dist
0,2020-10-01,2020,10,1,2020-2021,Happy,11,#FFD966,LEGEND_EXACT,0.0
1,2020-10-02,2020,10,2,2020-2021,Stress/Anxiety,5,#D1802C,LEGEND_EXACT,0.0
2,2020-10-03,2020,10,3,2020-2021,Depressed,3,#1155CC,LEGEND_EXACT,0.0
3,2020-10-04,2020,10,4,2020-2021,Depressed,3,#1155CC,LEGEND_EXACT,0.0
4,2020-10-05,2020,10,5,2020-2021,Tired,8,#9FC5E8,LEGEND_EXACT,0.0
5,2020-10-06,2020,10,6,2020-2021,Tired,8,#9FC5E8,LEGEND_EXACT,0.0
6,2020-10-07,2020,10,7,2020-2021,Good,9,#93C47D,LEGEND_EXACT,0.0
7,2020-10-08,2020,10,8,2020-2021,Good,9,#93C47D,LEGEND_EXACT,0.0
8,2020-10-09,2020,10,9,2020-2021,Depressed,3,#1155CC,LEGEND_EXACT,0.0
9,2020-10-10,2020,10,10,2020-2021,Hopeless,2,#674EA7,LEGEND_EXACT,0.0


In [22]:
unique_days = df_all["date"].nunique()
min_date = df_all["date"].min()
max_date = df_all["date"].max()

print("Unique extracted days:", unique_days)
print("Date range:", min_date.date(), "→", max_date.date())


Unique extracted days: 1899
Date range: 2020-10-01 → 2025-12-12


In [23]:
# Validate row counts for 2023

df_all[df_all["year"] == 2023]["date"].nunique()

# Expected: close to 365 (or exactly 365 if every valid day has a color).

365

In [24]:
# --- Re-run extraction from scratch (forces df_all to rebuild) ---

df_2020, df_2021, df_2022, unmapped_bridge = extract_composed_years_2020_2022()

dfs_single = []
unmapped_single = {}

for tab, cfg in SINGLE_YEAR_TABS.items():
    grid_range = get_effective_range(tab, cfg["range"])
    df_y, um = extract_single_year_fixed(tab, grid_range, cfg["year"])
    dfs_single.append(df_y)
    if um:
        unmapped_single[tab] = um

df_2023_2025 = pd.concat(dfs_single, ignore_index=True)

df_all = (
    pd.concat([df_2020, df_2021, df_2022, df_2023_2025], ignore_index=True)
    .sort_values("date")
    .reset_index(drop=True)
)

print("Rebuilt df_all rows:", len(df_all))
print("Unique days:", df_all["date"].nunique())
print(df_all.groupby("year")["date"].nunique())


Rebuilt df_all rows: 1899
Unique days: 1899
year
2020     92
2021    365
2022    365
2023    365
2024    366
2025    346
Name: date, dtype: int64


In [25]:
import calendar
import pandas as pd

def missing_dates_for_year(df, year):
    start = pd.Timestamp(year=year, month=1, day=1)
    end = pd.Timestamp(year=year, month=12, day=31)
    full = pd.date_range(start, end, freq="D")
    present = pd.to_datetime(df[df["year"] == year]["date"])
    missing = full.difference(present)
    return missing

for y in sorted(df_all["year"].unique()):
    miss = missing_dates_for_year(df_all, y)
    print(y, "missing:", len(miss))
    if len(miss) > 0:
        print(list(miss.date)[:20])


2020 missing: 274
[datetime.date(2020, 1, 1), datetime.date(2020, 1, 2), datetime.date(2020, 1, 3), datetime.date(2020, 1, 4), datetime.date(2020, 1, 5), datetime.date(2020, 1, 6), datetime.date(2020, 1, 7), datetime.date(2020, 1, 8), datetime.date(2020, 1, 9), datetime.date(2020, 1, 10), datetime.date(2020, 1, 11), datetime.date(2020, 1, 12), datetime.date(2020, 1, 13), datetime.date(2020, 1, 14), datetime.date(2020, 1, 15), datetime.date(2020, 1, 16), datetime.date(2020, 1, 17), datetime.date(2020, 1, 18), datetime.date(2020, 1, 19), datetime.date(2020, 1, 20)]
2021 missing: 0
2022 missing: 0
2023 missing: 0
2024 missing: 0
2025 missing: 19
[datetime.date(2025, 12, 13), datetime.date(2025, 12, 14), datetime.date(2025, 12, 15), datetime.date(2025, 12, 16), datetime.date(2025, 12, 17), datetime.date(2025, 12, 18), datetime.date(2025, 12, 19), datetime.date(2025, 12, 20), datetime.date(2025, 12, 21), datetime.date(2025, 12, 22), datetime.date(2025, 12, 23), datetime.date(2025, 12, 24), 

## Optional: Debug Unmapped Colors

In [26]:
import calendar

def month_to_col_letter(month: int) -> str:
    # Month columns for SINGLE_YEAR_TABS are B..M (12 columns)
    # month=1 -> B, month=12 -> M
    return chr(ord('B') + (month - 1))

def debug_single_year_cell(sheet: str, year: int, month: int, day: int):
    col = month_to_col_letter(month)
    a1 = f"{col}{day+1}:{col}{day+1}"  # day rows start at row 2 -> day 1 is row 2
    rng = f"{quote_sheet_name(sheet)}!{a1}"

    resp = service.spreadsheets().get(
        spreadsheetId=SPREADSHEET_ID,
        ranges=[rng],
        includeGridData=True
    ).execute()

    cell = resp["sheets"][0]["data"][0]["rowData"][0]["values"][0]
    fmt = cell.get("effectiveFormat", {})

    print("A1:", rng)
    print("hex_from_cell:", hex_from_cell(cell))
    print("backgroundColor:", fmt.get("backgroundColor"))
    print("backgroundColorStyle:", fmt.get("backgroundColorStyle"))
    print("userEnteredFormat.bg:", (cell.get("userEnteredFormat") or {}).get("backgroundColor"))
    print("userEnteredFormat.bgStyle:", (cell.get("userEnteredFormat") or {}).get("backgroundColorStyle"))


In [27]:
debug_single_year_cell(sheet="2023", year=2023, month=3, day=20)


A1: '2023'!D21:D21
hex_from_cell: #000000
backgroundColor: {}
backgroundColorStyle: {'themeColor': 'TEXT'}
userEnteredFormat.bg: {}
userEnteredFormat.bgStyle: {'themeColor': 'TEXT'}


# Show Metrics

In [29]:
# Monthly Happiness Index (HI)
monthly_hi = (
    df_all.groupby(["year", "month"], as_index=False)["score"]
    .sum()
    .rename(columns={"score": "happiness_index"})
    .sort_values(["year", "month"])
)

# Yearly totals per emotion + % of days
year_emotion = (
    df_all.groupby(["year", "emotion"], as_index=False)
    .agg(days=("date", "count"), total_score=("score", "sum"))
)

year_totals = df_all.groupby("year", as_index=False).agg(
    total_days=("date", "count"),
    total_score=("score", "sum")
)

year_emotion = year_emotion.merge(year_totals, on="year", how="left")
year_emotion["pct_days"] = (year_emotion["days"] / year_emotion["total_days"]) * 100

# Total Emotional Index across extracted days
# (Uses extracted days only; if some days are blank, TEI reflects that.)
theoretical_max = 11 * df_all["date"].nunique()
tei = (df_all["score"].sum() / theoretical_max) if theoretical_max else None

print("\n--- Monthly HI (preview) ---")
display(monthly_hi.head(24))

print("\n--- Yearly emotion breakdown (preview) ---")
display(year_emotion.sort_values(["year", "emotion"]).head(60))

print("\n--- Total Emotional Index (across extracted days) ---")
print("theoretical_max =", theoretical_max)
print("total_score     =", int(df_all["score"].sum()))
print("TEI             =", tei)



--- Monthly HI (preview) ---


,year,month,happiness_index
0,2020,10,181
1,2020,11,232
2,2020,12,216
3,2021,1,222
4,2021,2,159
5,2021,3,234
6,2021,4,227
7,2021,5,221
8,2021,6,214
9,2021,7,212



--- Yearly emotion breakdown (preview) ---


,year,emotion,days,total_score_x,total_days,total_score_y,pct_days
0,2020,Angry/Annoyed,5,20,92,629,5.434783
1,2020,Depressed,16,48,92,629,17.391304
2,2020,Good,22,198,92,629,23.913043
3,2020,Happy,4,44,92,629,4.347826
4,2020,Hopeless,1,2,92,629,1.086957
5,2020,Lazy,13,91,92,629,14.130435
6,2020,Productive,12,120,92,629,13.043478
7,2020,SAD,2,12,92,629,2.173913
8,2020,Stress/Anxiety,7,35,92,629,7.608696
9,2020,Suicidal,3,3,92,629,3.260870



--- Total Emotional Index (across extracted days) ---
theoretical_max = 20889
total_score     = 14435
TEI             = 0.6910335583321365


In [30]:
monthly_hi.query("year == 2020 and month in [10,11,12]")


,year,month,happiness_index
0,2020,10,181
1,2020,11,232
2,2020,12,216


In [31]:
(df_all[(df_all.year == 2020) & (df_all.month == 10)]
 .groupby(["emotion"], as_index=False)
 .agg(days=("date","count"), total=("score","sum"))
 .sort_values("total", ascending=False))


,emotion,days,total
2,Good,6,54
9,Tired,6,48
1,Depressed,7,21
5,Lazy,2,14
3,Happy,1,11
7,Stress/Anxiety,2,10
6,Productive,1,10
0,Angry/Annoyed,2,8
8,Suicidal,3,3
4,Hopeless,1,2


In [40]:
import re
import pandas as pd

# -----------------------------
# Helpers for month header parsing
# -----------------------------
MONTH_NAME_TO_NUM = {
    "january": 1, "february": 2, "march": 3, "april": 4, "may": 5, "june": 6,
    "july": 7, "august": 8, "september": 9, "october": 10, "november": 11, "december": 12
}

def parse_month_header(text: str):
    """
    Parses headers like:
      "January 2023", "February", "October 2020", "November", etc.
    Returns (month_num, year_or_none) or None if can't parse.
    """
    if text is None:
        return None
    s = str(text).strip()
    if not s:
        return None

    m = re.match(r"^(January|February|March|April|May|June|July|August|September|October|November|December)\s*(\d{4})?$", s, re.I)
    if not m:
        return None

    month_num = MONTH_NAME_TO_NUM[m.group(1).lower()]
    year = int(m.group(2)) if m.group(2) else None
    return month_num, year


def grid_to_header_range(grid_range: str):
    """
    Convert something like:
      B45:M45 -> B1:M1
    (we only need the columns; row number doesn't matter as long as it contains month labels)
    """
    start, end = grid_range.split(":")
    start_col = re.match(r"([A-Z]+)\d+", start).group(1)
    end_col = re.match(r"([A-Z]+)\d+", end).group(1)
    return f"{start_col}1:{end_col}1"


def fetch_row_values(sheet_name: str, a1_range: str):
    """
    Fetch a single row range like B45:M45 as values.
    Returns list of values (length = number of columns).
    """
    vals = fetch_values(sheet_name, a1_range)
    if not vals:
        return []
    return vals[0]


def fetch_headers(sheet_name: str, header_range: str):
    """
    Fetch header labels for the same columns as the HI row.
    Returns list of header strings.
    """
    vals = fetch_values(sheet_name, header_range)
    if not vals:
        return []
    return vals[0]


# -----------------------------
# Main: build monthly HI from sheet-calculated values
# -----------------------------
def build_monthly_hi_from_sheet():
    """
    Returns DataFrame with:
      year, month, happiness_index, source_sheet
    """

    records = []

    # 1) Single-year tabs (Jan..Dec): values in B45:M45, headers in B1:M1
    single_tabs = {
        "2023": {"hi_range": "B45:M45"},
        "2024": {"hi_range": "B45:M45"},
        "2025": {"hi_range": "B45:M45"},
    }

    for tab, cfg in single_tabs.items():
        hi_range = cfg["hi_range"]
        header_range = grid_to_header_range(hi_range)

        headers = fetch_headers(tab, header_range)
        his = fetch_row_values(tab, hi_range)

        for h, v in zip(headers, his):
            parsed = parse_month_header(h)
            if not parsed:
                continue
            month_num, year_in_header = parsed

            # For single-year tabs, the year should be the tab year
            year = int(tab)

            # numeric HI cell value
            if v is None or str(v).strip() == "":
                continue
            hi_val = float(v)

            records.append({
                "year": year,
                "month": month_num,
                "happiness_index": hi_val,
                "source_sheet": tab
            })

    # 2) Bridge tabs (need year from header if present, otherwise infer)
    #    2020-2021: B45:N45 (Oct 2020 .. Oct 2021)
    #    2021-2022: B45:O45 (Nov 2021 .. Dec 2022)
    bridge_tabs = {
        "2020-2021": {"hi_range": "B45:N45"},
        "2021-2022": {"hi_range": "B45:O45"},
    }

    for tab, cfg in bridge_tabs.items():
        hi_range = cfg["hi_range"]
        header_range = grid_to_header_range(hi_range)

        headers = fetch_headers(tab, header_range)
        his = fetch_row_values(tab, hi_range)

        # We'll infer missing years by carrying forward the last seen year
        current_year = None

        for h, v in zip(headers, his):
            parsed = parse_month_header(h)
            if not parsed:
                continue
            month_num, year_in_header = parsed

            if year_in_header is not None:
                current_year = year_in_header
            elif current_year is None:
                # If first header has no year, we can't infer safely
                # (your first headers DO have years like "October 2020" / "November 2021")
                continue

            year = current_year

            if v is None or str(v).strip() == "":
                continue
            hi_val = float(v)

            records.append({
                "year": year,
                "month": month_num,
                "happiness_index": hi_val,
                "source_sheet": tab
            })

    monthly_hi_sheet = (
        pd.DataFrame(records)
        .drop_duplicates(subset=["year", "month"], keep="last")
        .sort_values(["year", "month"])
        .reset_index(drop=True)
    )
    return monthly_hi_sheet



monthly_hi_sheet = build_monthly_hi_from_sheet()
monthly_hi_sheet["happiness_index"] = monthly_hi_sheet["happiness_index"].astype(int)
print("Total monthly HI rows:", len(monthly_hi_sheet))
display(monthly_hi_sheet.head(30))


Total monthly HI rows: 63


,year,month,happiness_index,source_sheet
0,2020,10,146,2020-2021
1,2020,11,211,2020-2021
2,2020,12,190,2020-2021
3,2021,1,187,2020-2021
4,2021,2,132,2020-2021
5,2021,3,211,2020-2021
6,2021,4,203,2020-2021
7,2021,5,221,2020-2021
8,2021,6,214,2020-2021
9,2021,7,207,2020-2021


# Save Outputs (for dashboard)

In [41]:
df_all.to_csv("mood_all_years.csv", index=False)
monthly_hi_sheet.to_csv("mood_monthly_hi_sheet.csv", index=False)
year_emotion.to_csv("mood_year_emotion_breakdown.csv", index=False)

print("\nSaved files:")
print("- mood_all_years.csv")
print("- mood_monthly_hi.csv")
print("- mood_year_emotion_breakdown.csv")


Saved files:
- mood_all_years.csv
- mood_monthly_hi.csv
- mood_year_emotion_breakdown.csv


In [ ]:
366+365+427+393+344

1895

In [42]:
monthly_hi_sheet.to_csv("mood_monthly_hi_sheet.csv", index=False)